In [1]:
import ee
import geemap as emap
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

    

Enter verification code: 4/1AX4XfWhxY6JIlkuKg6ivnYpCkj_5eHvWJaMIYr6UOvEr7hQWFtUbN772aL4

Successfully saved authorization token.


In [2]:
states = ee.FeatureCollection('TIGER/2018/States').sort('ALAND', False)

sd=states.flatten()
try:
    sd=states.fatten()
    print(sd)
except:
    print("Error occurred")


Error occurred


In [3]:
# Fetch a digital elevation model.
image = ee.Image('CGIAR/SRTM90_V4')

# Request a default thumbnail of the DEM with defined linear stretch.
# Set masked pixels (ocean) to 1000 so they map as gray.
thumbnail1 = image.unmask(1000).getThumbURL({
  'min': 0,
  'max': 3000,
  'dimensions': 500
})
print('Default extent and size:', thumbnail1)

# Specify region by GeoJSON, define palette, set size of the larger aspect dimension.
thumbnail2 = image.getThumbURL({
  'min': 0,
  'max': 3000,
  'palette': ['00A600','63C600','E6E600','E9BD3A','ECB176','EFC2B3','F2F2F2'],
  'dimensions': 500,
  'region': ee.Geometry.Rectangle([-84.6, -55.9, -32.9, 15.7]),
})
print('GeoJSON region, palette, and max dimension:', thumbnail2)

Default extent and size: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3bbe407c4fb4d95e5a03d51383de3238-1b298ef662ea32faa7db8c0e6d5e49a6:getPixels
GeoJSON region, palette, and max dimension: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c422a98c58645c378ee7513b126ef150-a883305befbbcc3e808d5a71ee178094:getPixels


## Display GEE Base map

In [27]:
mymap=emap.Map()
mymap

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# Display USA Map

In [32]:
usa_map=ee.Image().paint(states,0,2)

mymap.addLayer(usa_map,{"palette":'blue'},"USA")

### 1. Add a new column to attribute table?

In [34]:
states = ee.FeatureCollection('TIGER/2018/States')
states = states.map(lambda x: x.set('AreaKm2', x.area().divide(1000000).toLong()))
first = states.first().toDictionary().getInfo()
print(first)

{'ALAND': 348021896, 'AWATER': 1550236201, 'AreaKm2': 1904, 'DIVISION': '0', 'FUNCSTAT': 'A', 'GEOID': '78', 'INTPTLAT': '+18.3267480', 'INTPTLON': '-064.9712508', 'LSAD': '00', 'MTFCC': 'G4000', 'NAME': 'United States Virgin Islands', 'REGION': '9', 'STATEFP': '78', 'STATENS': '01802710', 'STUSPS': 'VI'}


In [91]:
myflat=states.flatten()

print(myflat)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.flatten",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.loadTable",
          "arguments": {
            "tableId": {
              "constantValue": "TIGER/2018/States"
            }
          }
        }
      }
    }
  }
})


In [92]:
print(states)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.loadTable",
    "arguments": {
      "tableId": {
        "constantValue": "TIGER/2018/States"
      }
    }
  }
})


## Display shapefiles based on a column value

In [55]:
states = ee.FeatureCollection('TIGER/2018/States')

visParams={"palette":["green","yellow","orange","blue","red"],"min":158340391,"max":1478839695958}

paint=ee.Image().toFloat().paint(states,"ALAND")
mymap.addLayer(paint,visParams,"MAP")

In [42]:
mymap

Map(bottom=1039.800033569336, center=[41.40317929023851, -71.54296875000001], controls=(WidgetControl(options=…

## Download attribute table as csv file

In [63]:
states=states = ee.FeatureCollection('TIGER/2018/States')
url=states.getDownloadURL(filetype="csv",selectors=["ALAND","AWATER","NAME","STUSPS"],filename="MyCSV") # If one wants to downalod all columns, leave selectors 
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/1994b38b7b54f2dd9178debead98d099-1786023cadf6fcfa48723c01199db7e4:getFeatures


## Display column names

In [70]:
col_names = states.first().propertyNames().getInfo()
print("Column names: ", col_names)

Column names:  ['STATENS', 'GEOID', 'AWATER', 'LSAD', 'STUSPS', 'STATEFP', 'FUNCSTAT', 'INTPTLAT', 'DIVISION', 'REGION', 'NAME', 'INTPTLON', 'MTFCC', 'ALAND', 'system:index']


In [71]:
states.size().getInfo()

56

In [89]:
Map = emap.Map()

#Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

#Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, 10) # San Francisco Bay
Map.addLayer(ndwi, ndwiViz, 'NDWI', False)

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked')

# Display the map
Map.addLayerControl()
Map

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

In [97]:
Map = emap.Map()

#Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

#Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, 10) # San Francisco Bay

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4));

# Create visualization layers.
imageRGB = image.visualize(**{'bands': ['B5', 'B4', 'B3'], 'max': 0.5})
ndwiRGB = ndwiMasked.visualize(**{
  'min': 0.5,
  'max': 1,
  'palette': ['00FFFF', '0000FF']
})

# Mosaic the visualization layers and display (or export).
mosaic = ee.ImageCollection([imageRGB, ndwiRGB]).mosaic()
Map.addLayer(mosaic, {}, 'mosaic');
Map.addLayer(imageRGB,{},"HND")
# Display the map
Map.addLayerControl()
Map

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…